# Import thư viện

In [1]:
import requests
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path

# Tạo thư mục để lưu dữ liệu

In [2]:
# ------------------- Thư mục lưu -------------------
OUT = Path(r"C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw")
OUT.mkdir(parents=True, exist_ok=True)


# Lấy dữ liệu trên Yahoo bằng API

In [3]:
# ------------------- Yahoo Chart API -------------------
def yahoo_chart(
    symbol: str,                # Mã cổ phiếu hoặc chỉ số cần lấy (VD: "VNINDEX", "AAPL", "BTC-USD")
    start_date="2015-01-01",    # Ngày bắt đầu lấy dữ liệu (ISO format: YYYY-MM-DD)
    interval="1d",              # Khoảng thời gian giữa các điểm dữ liệu ("1d" = theo ngày)
    close_only=True             # Nếu True → chỉ lấy giá đóng cửa; False → lấy đủ OHLCV
) -> pd.DataFrame:
    """
    Crawl (thu thập) dữ liệu thô từ Yahoo Finance Chart API.
    Trả về DataFrame chứa dữ liệu giá theo thời gian.
    Có thể chọn lấy chỉ giá đóng cửa (close_only=True) hoặc toàn bộ OHLC.
    """

    # ==========================================================
    # 1️⃣ TÍNH THỜI GIAN EPOCH (tính bằng giây kể từ 1970-01-01 UTC)
    # ==========================================================
    dt1 = datetime.fromisoformat(start_date)                 # Chuyển chuỗi start_date thành đối tượng datetime
    period1 = int(dt1.replace(tzinfo=timezone.utc).timestamp())  # Chuyển datetime → epoch (giây UTC)
    period2 = int(datetime.now(tz=timezone.utc).timestamp())     # Lấy thời gian hiện tại (epoch) làm mốc kết thúc

    # ==========================================================
    # 2️⃣ GỬI REQUEST ĐẾN YAHOO FINANCE API
    # ==========================================================
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}"  # API endpoint
    params = {
        "period1": period1,             # Thời điểm bắt đầu (epoch)
        "period2": period2,             # Thời điểm kết thúc (epoch)
        "interval": interval,           # Khoảng cách dữ liệu ("1d" = daily, "1wk" = weekly, ...)
        "includePrePost": "true",       # Bao gồm dữ liệu ngoài giờ (nếu có)
        "events": "div,splits",         # Bao gồm cổ tức (dividends) và chia tách (splits)
    }

    # Gửi yêu cầu HTTP GET tới API với header giả lập trình duyệt thật
    r = requests.get(
        url,
        params=params,
        headers={"User-Agent": "Mozilla/5.0"},
        timeout=30
    )

    # Nếu HTTP trả lỗi (404, 500, ...) → raise exception
    r.raise_for_status()

    # ==========================================================
    # 3️⃣ PHÂN TÍCH JSON TRẢ VỀ (PARSE JSON → DataFrame)
    # ==========================================================
    result = r.json().get("chart", {}).get("result")  # Lấy phần dữ liệu chính từ JSON
    if not result:
        # Nếu không có dữ liệu → báo lỗi
        raise RuntimeError("Không có dữ liệu trả về từ Yahoo API.")

    node = result[0]                     # Yahoo luôn trả về list chứa 1 node chính
    ts = node.get("timestamp", [])       # Danh sách timestamp (giây epoch)
    ind = node.get("indicators", {})     # Phần chứa các chỉ số giá
    quote = (ind.get("quote") or [{}])[0]    # Truy cập dictionary chứa open, high, low, close, volume
    adj   = (ind.get("adjclose") or [{}])[0] # Dữ liệu giá điều chỉnh (adjusted close)

    # ==========================================================
    # 4️⃣ CHUYỂN DỮ LIỆU THÀNH PANDAS DATAFRAME
    # ==========================================================
    df = pd.DataFrame({
        # Chuyển từng timestamp thành dạng datetime UTC
        "datetime_utc": [datetime.fromtimestamp(t, tz=timezone.utc) for t in ts],
        "close": quote.get("close"),     # Giá đóng cửa
    })

    # Nếu cần dữ liệu đầy đủ (close_only = False) → thêm các cột khác
    if not close_only:
        df["open"] = quote.get("open")       # Giá mở cửa
        df["high"] = quote.get("high")       # Giá cao nhất
        df["low"] = quote.get("low")         # Giá thấp nhất
        df["volume"] = quote.get("volume")   # Khối lượng giao dịch
        df["adjclose"] = adj.get("adjclose") # Giá đóng cửa đã điều chỉnh

    # Trả về DataFrame hoàn chỉnh
    return df


# Dữ liệu cần thiết để crawl về và lưu dữ liệu tại:
-Lưu: C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw\VND_X_1d.csv

-Lưu: C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw\BZ_F_1d.csv

In [4]:
# ------------------- Main Crawl -------------------
if __name__ == "__main__":
    symbols = [
        ("VND=X", "2015-01-01", "1d"),
        ("BZ=F", "2015-01-01", "1d"),
       
    ]

    for sym, start, interval in symbols:
        try:
            df = yahoo_chart(sym, start_date=start, interval=interval, close_only=True)
            out_path = OUT / f"{sym.replace('=','_').replace('^','')}_{interval}.csv"
            df.to_csv(out_path, index=False, encoding="utf-8-sig")
            print(f"✅ {sym} ({interval}) -> {len(df)} dòng | Lưu: {out_path}")
        except Exception as e:
            print(f"❌ Lỗi {sym}: {e}")

❌ Lỗi VND=X: HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Read timed out. (read timeout=30)
✅ BZ=F (1d) -> 2730 dòng | Lưu: C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw\BZ_F_1d.csv
